In [25]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [10]:
%run -i ../../zkstats/core.py

In [11]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [24]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

# make sure dummy is not 0 due to harmonic mean
dummy_data = np.round(np.random.uniform(1,10,len(data)),1)
# print("dummy: ", dummy_data)
# dummy_data = np.random.uniform(min(data), max(data), len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.div(1.0,torch.mean(torch.div(1.0,dummy_data_tensor)))

dummy:  [ 1.1  7.7  8.6  4.2  2.7  1.3  1.3  1.7  6.6  7.3  9.4  5.5  7.8  3.2
  1.5  4.4  6.6  7.5  6.1 10.   9.1  9.3  5.7  8.6  5.9  7.8  4.9  3.8
  2.9  9.8  7.7  2.5  8.2  4.4  1.8  6.3  2.8  2.2  7.8  2.6  9.5  9.8
  4.8  3.   8.5  7.3  8.4  6.5  2.4  1.7  1.7  6.6  8.   9.7  8.2  2.5
  5.3  9.   1.8  4.6  2.6  8.   2.8  7.4  1.8  1.4  6.1  6.7  1.6  5.
  8.   9.4  9.1  4.2  7.7  4.7  1.4  2.   8.   7.6  9.5  8.1  8.4  1.3
  4.8  4.1  5.9  4.5  8.9  4.1  4.9  6.9  4.3  1.1  2.5  3.5  7.5  5.5
  5.3  1.4  8.4  5.1  7.1  1.8  7.2  4.1  7.2  1.9  6.2  8.6  7.6  8.5
  6.5  7.1  5.7  1.3  8.2  2.3  1.5  4.3  8.2  6.9  1.2  4.7  1.   4.7
  8.3  7.4  3.6  8.2  6.4  1.6  2.3  7.4  8.   4.2  9.4  3.   6.6  1.7
  1.8  3.   3.2  6.7  9.1  3.8  6.1  7.3  2.3  8.6  4.2  8.2  3.6  7.4
  2.3  6.8  3.2  4.9  8.6  9.1  7.3  9.5  6.3  4.5  8.2  9.4  3.7  1.6
  9.8  9.8  7.4  5.4  4.1  5.4  3.3  4.5  1.7  5.4  9.5  2.2  6.1  8.3
  6.5  1.7  7.5  8.2  2.6  5.6  3.6  5.7  7.3  5.5  8.   5.2  8.5  2.


In [25]:
# Verifier/ data consumer side:
# since have division, scale must be very LARGE, cant use just 0
# So, we allow error rate to be 10% instead of 1%
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.abs((self.w*torch.sum(torch.div(1.0,X)))-X.size()[1])<=torch.abs(0.1*X.size()[1]), self.w)
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

In [26]:
# prover calculates settings, send to verifier
theory_output = torch.div(1.0,torch.mean(torch.div(1.0,data_tensor)))
print("Theory_output: ",theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.abs((self.w*torch.sum(torch.div(1.0,X)))-X.size()[1])<=torch.abs(0.1*X.size()[1]), self.w)

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [6], "resources", settings_path)

Theory_output:  tensor(45.2144)
==== Generate & Calibrate Setting ====
scale:  [6]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":6,"param_scale":6,"scale_rebase_multiplier":10,"lookup_range":[-184392,11264],"logrows":18,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":911,"total_const_size":4,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,6],"model_input_scales":[6],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Recip":{"scale":4096.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.4.2","num_blinding_factors":null}


In [27]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 26.29948902130127 seconds
Theory output:  tensor(45.2144)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 45.21875
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[11712583131456230033, 7998639690917952521, 17628064685104548320, 3364894255067083796], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [10409296726488761395, 15557203226960814343, 16692342436085548322, 2420882828764455196]]], 'proof': '214b1c345807dda0e132d0c0340e9e7b828152907a40f51f41ca1d13e855f0830edf53504c6e01897dad2f06a99d1c86d1ec85ee36c6298c3afd9e16bf83abbb08116d09cc305949d5dd4d323b3a83168dc5988448a473c7d0866e5acb5b4693278fbc4ab9706e3944aaf44d923cc41cbb3f1df34bc840caa00cabb2fe0867750494e14c682da7450caf73d80a8502d55c35f895e3ca5ce8d330ef1cf4965a3618dcfb4fe048422474b11dd9916f544dcf792f9c8cc5249d75798ecaeec4cbe20648279767c80b2211c942b23d4d56d98486122792480a965f0dde722a0ff3591d4afacd00ff2ed52078c0d2d0eccc8af046ac3dccfbccc070b69891164af5ae2b73901340995f94c14d7eab0ebaf69f1ab3f3084901046972f8c9142f5ad91b1ee5fba0f853fb759aa9e567506892d16ce801ce10b3d056aced17ab4a84334d17a2c82ed1c742cada3f8a4aaad5da35decdba4dc009a3b3db24cd753ce7

In [28]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[11712583131456230033, 7998639690917952521, 17628064685104548320, 3364894255067083796], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [10409296726488761395, 15557203226960814343, 16692342436085548322, 2420882828764455196]]]
proof boolean:  1.0
proof result 1 : 45.21875
verified
